# Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import pickle
from sklearn.preprocessing import StandardScaler

# Main tables

In [ ]:
df = pd.read_parquet("./../SIBUR_2023/train.parquet")
df['target_class'] = (df['feature4'] == 'gas1') * 1
df.drop(labels=['feature4'], axis=1, inplace=True)
key_cols = ['target0', 'target1', 'target_class']
cols_for_features = ['feature' + str(i) for i in range(25) if i != 4]

_____

_______

# Models for target0 gas=2

###### Ridge

In [ ]:
current_target = 'target0'
current_target_class = 0
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=4)),
                  ('ss', StandardScaler(with_std=True)),
                  ('lasso', Ridge(alpha=5))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '_big.pkl', 'wb'))

_____

# Models for target0 gas=1

###### Lasso

In [ ]:
current_target = 'target0'
current_target_class = 1
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
                  ('model_ridge', Lasso(alpha=1))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '.pkl', 'wb'))

###### SVR

In [ ]:
%%time
current_target = 'target0'
current_target_class = 1
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('model_ridge', SVR(kernel='poly', degree=4, C=3.0))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '_svr_d4_C3.pkl', 'wb'))

______

# Models for target1 gas=2

###### Lasso

In [ ]:
current_target = 'target1'
current_target_class = 0
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
                  ('model_ridge', Lasso(alpha=1))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '.pkl', 'wb'))

###### Ridge

In [ ]:
current_target = 'target1'
current_target_class = 0
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
                  ('ss', StandardScaler(with_std=True)),
                  ('lasso', Ridge(alpha=0.01))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '_ss_ridge_001.pkl', 'wb'))

______

# Models for target1 gas=1

###### SVR

In [ ]:
%%time
current_target = 'target1'
current_target_class = 1
current_train = df[df['target_class'] == current_target_class].reset_index(drop=True)
model = Pipeline([('model_ridge', SVR(kernel='poly', degree=5, C=3.0))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '_svr_C_3.pkl', 'wb'))

###### Lasso squares

In [ ]:
%%time
current_target = 'target1'
current_target_class = 1
train11 = df.loc[df['target_class'] == current_target_class, :].reset_index(drop=True)
train11_target = train11[current_target].values
train11 = train11[cols_for_features]
for i in range(len(cols_for_features)):
    train11[cols_for_features[i] + '_2'] = train11[cols_for_features[i]] * train11[cols_for_features[i]]
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
                  ('ss', StandardScaler()),
                  ('lasso', Lasso(alpha=0.0001))])
model.fit(train11, train11_target)
pickle.dump(model,
    open('model_' + current_target + '_for_target_' + str(current_target_class) + '_lasso_squares.pkl', 'wb'))

_____

_____

_____

# Create predict.py

In [ ]:
%%writefile predict.py

import warnings
warnings.filterwarnings('ignore')

import pathlib
import pandas as pd
import numpy as np
import pickle
from multiprocessing import Pool

DATA_DIR = pathlib.Path(".")

# Lasso
MODEL_FILE_target01 = pathlib.Path(__file__).parent.joinpath("model_target0_for_target_1.pkl")
MODEL_FILE_target10 = pathlib.Path(__file__).parent.joinpath("model_target1_for_target_0.pkl")

# Ridge
MODEL_FILE_target00_ridge = pathlib.Path(__file__).parent.joinpath("model_target0_for_target_0_big.pkl")
MODEL_FILE_target10_ridge = pathlib.Path(__file__).parent.joinpath("model_target1_for_target_0_ss_ridge_001.pkl")

# SVR
MODEL_FILE_target01_svr = pathlib.Path(__file__).parent.joinpath("model_target0_for_target_1_svr_d4_C3.pkl")
MODEL_FILE_target11_svr = pathlib.Path(__file__).parent.joinpath("model_target1_for_target_1_svr_C_3.pkl")

# Square
MODEL_FILE_target11_square = pathlib.Path(__file__).parent.joinpath("model_target1_for_target_1_lasso_squares.pkl")


models_dict = {}
models_dict['target0'] = {}
models_dict['target1'] = {}

models_dict['target0'][0] = {}
models_dict['target0'][0]['ridge'] = MODEL_FILE_target00_ridge

models_dict['target0'][1] = {}
models_dict['target0'][1]['lasso'] = MODEL_FILE_target01
models_dict['target0'][1]['svr'] = MODEL_FILE_target01_svr

models_dict['target1'][0] = {}
models_dict['target1'][0]['lasso'] = MODEL_FILE_target10
models_dict['target1'][0]['ridge'] = MODEL_FILE_target10_ridge

models_dict['target1'][1] = {}
models_dict['target1'][1]['svr'] = MODEL_FILE_target11_svr
models_dict['target1'][1]['square'] = MODEL_FILE_target11_square

def predict_batch(df: pd.DataFrame) -> pd.DataFrame:
    
    df['target_class'] = (df['feature4'] == 'gas1') * 1
    df.drop(labels=['feature4'], axis=1, inplace=True)
    key_cols = ['target0', 'target1', 'target_class']
    cols_for_features = ['feature' + str(i) for i in range(25) if i != 4]
    
    predictions = df[['target_class']]
    
    
    current_target = 'target0'
    current_target_class = 0
    current_test = df[df['target_class'] == current_target_class]
    model_ridge = pickle.load(open(models_dict[current_target][current_target_class]['ridge'], 'rb'))
    predictions.loc[current_test.index, current_target] = model_ridge.predict(current_test[cols_for_features])
    
    
    current_target = 'target0'
    current_target_class = 1
    current_test = df[df['target_class'] == current_target_class]
    model_lasso = pickle.load(open(models_dict[current_target][current_target_class]['lasso'], 'rb'))
    model_svr = pickle.load(open(models_dict[current_target][current_target_class]['svr'], 'rb'))
    predictions.loc[current_test.index, current_target] =\
                            (0.5 * model_lasso.predict(current_test[cols_for_features]) +\
                             0.5 * model_svr.predict(current_test[cols_for_features]))
    
    
    current_target = 'target1'
    current_target_class = 0
    current_test = df[df['target_class'] == current_target_class]
    model_lasso = pickle.load(open(models_dict[current_target][current_target_class]['lasso'], 'rb'))
    model_ridge = pickle.load(open(models_dict[current_target][current_target_class]['ridge'], 'rb'))
    predictions.loc[current_test.index, current_target] =\
                            (0.5 * model_lasso.predict(current_test[cols_for_features]) +\
                             0.5 * model_ridge.predict(current_test[cols_for_features]))
    
    
    current_target = 'target1'
    current_target_class = 1
    current_test = df.loc[df['target_class'] == current_target_class]
    model_svr = pickle.load(open(models_dict[current_target][current_target_class]['svr'], 'rb'))
    
    model_square = pickle.load(open(models_dict[current_target][current_target_class]['square'], 'rb'))
    test11 = df.loc[df['target_class'] == current_target_class, cols_for_features]
    for i in range(len(cols_for_features)):
        test11[cols_for_features[i] + '_2'] = test11[cols_for_features[i]] * test11[cols_for_features[i]]
        
    predictions.loc[current_test.index, current_target] =\
                            (0.5 * model_square.predict(test11) +\
                             0.5 * model_svr.predict(current_test[cols_for_features]))

    
    predictions = predictions[['target0', 'target1']]
    predictions['target0'] = np.clip(predictions['target0'], 0, 100)
    predictions['target1'] = np.clip(predictions['target1'], 0, 100)
    
    return predictions

def predict(df: pd.DataFrame) -> pd.DataFrame:
    pool = Pool(4)
    prediction_table = pool.map(predict_batch,
                    [df.iloc[i * 5_000: (i + 1) * 5_000] for i in range(df.shape[0] // 5_000 + 1)])
    return pd.concat(prediction_table).sort_index()


_____

_____

_____

_____

_____

_____

_____

_____

_____